In [18]:
API_KEY = ''
BACKOFF_TIME = 30
RETURN_FULL_RESULTS = False

def get_google_results(address, api_key=None, return_full_response=False):
    
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address)
    geocode_url = geocode_url + "&key={}".format(api_key)
    results = requests.get(geocode_url)
    results = results.json()
    
    # if there's no results or an error, return empty results.
    if len(results['results']) == 0:
        output = {
            "formatted_address" : None,
            "latitude": None,
            "longitude": None,
            "accuracy": None,
            "google_place_id": None,
            "type": None,
            "postcode": None
        }
    else:    
        answer = results['results'][0]
        output = {
            "formatted_address" : answer.get('formatted_address'),
            "latitude": answer.get('geometry').get('location').get('lat'),
            "longitude": answer.get('geometry').get('location').get('lng'),
            "accuracy": answer.get('geometry').get('location_type'),
            "google_place_id": answer.get("place_id"),
            "type": ",".join(answer.get('types')),
            "postcode": ",".join([x['long_name'] for x in answer.get('address_components') 
                                  if 'postal_code' in x.get('types')])
        }
        
#     print(output['formatted_address'])
    # Append some other details:    
    output['input_string'] = address
    output['number_of_results'] = len(results['results'])
    output['status'] = results.get('status')
    if return_full_response is True:
        output['response'] = results
    
    return output

def geocode(address):
    # While the address geocoding is not finished:
    geocoded = False
    while geocoded is not True:
        # Geocode the address with google
        try:
            geocode_result = get_google_results(address, API_KEY, return_full_response=RETURN_FULL_RESULTS)
        except Exception as e:
            logger.exception(e)
            logger.error("Major error with {}".format(address))
            logger.error("Skipping!")
            geocoded = True
            
        # If we're over the API limit, backoff for a while and try again later.
        if geocode_result['status'] == 'OVER_QUERY_LIMIT':
            print("Hit Query Limit! Backing off for a bit.")
            time.sleep(BACKOFF_TIME * 60) # sleep for 30 minutes
            geocoded = False
        else:
            geocoded = True
#     print(geocode_result['formatted_address'])
    return geocode_result['formatted_address']

#     # Print status every 100 addresses
#     if len(results) % 100 == 0:
#     	logger.info("Completed {} of {} address".format(len(results), len(addresses)))
            
#     # Every 500 addresses, save progress to file(in case of a failure so you have something!)
#     if len(results) % 500 == 0:
#         pd.DataFrame(results).to_csv("{}_bak".format(output_filename))



In [19]:
# get google api data
df2 = df[df.zipcode_new=='']
df2 = df2.iloc[:10,:]

df2['street'] = df2.apply(lambda row: geocode(row.street) if row.zipcode_new=='' else row.street, axis=1)
df2[['street', 'street_cleansed', 'zipcode_new']].head(10)

,street,street_cleansed,zipcode_new
id,,,
13560665,None,grange,
10510447,None,NaN,
15610715,None,NaN,
14628174,None,NaN,
11086717,None,NaN,
15498190,None,NaN,
618124,None,NaN,
9722956,None,phoenix,
13712146,None,NaN,
